In [37]:
import numpy as np
from scipy.stats import norm,lognorm


gaussian copula for simulation of correlated assets to price multi asset derivatives

Let cor = correlation matrix.

generate $(x_1,x_2,...x_n)$ = multivariate_normal(zeros(n),cor,size=no of simulations)

transform each dimension $x_i$ using $asset_i = marginal^-1 (normal.cdf(x_i))$



In [38]:
rho = 0.3
cor = np.array([[1,rho],[rho,1]])
X = np.random.multivariate_normal(np.zeros(2), cor,size=100000)
# add -ve symmetric values for variance reduction
X = np.vstack((X,-X))
x0 = X[:,0]
x1 = X[:,1]
x0.mean(),x1.mean(),x0.std(),x1.std(),np.corrcoef(x0,x1)

(-3.375077994860476e-19,
 -9.237055564881302e-19,
 1.000791265172991,
 1.0035960671942024,
 array([[1.        , 0.30531643],
        [0.30531643, 1.        ]]))

marginal distribiution function is simply derivative of call option price with correct smile
We assume lognormal for simplicity
 
we use scale = $fwd_i*exp(-0.5\sigma_i^2t)$ and $s=\sigma_i*\sqrt{t}$ in the scipy lognorm distribution



In [39]:
feurusd= 1.13
veurusd = 0.1
faudusd= 0.69
vaudusd = 0.1
t = 1
# At this stage we should use the ppf of the marginal derived from the smile curve
eurusd = lognorm.ppf(norm.cdf(x0),veurusd,0,(feurusd*np.exp(-0.5*veurusd**2)))
audusd = lognorm.ppf(norm.cdf(x1),vaudusd,0,(faudusd*np.exp(-0.5*vaudusd**2)))
eurusd.mean(),audusd.mean(),np.std(np.log(eurusd)),np.std(np.log(audusd)),\
np.corrcoef(np.log(eurusd),np.log(audusd))



(1.1300088024740786,
 0.6900248437985322,
 0.10007912651729911,
 0.10035960671942022,
 array([[1.        , 0.30531643],
        [0.30531643, 1.        ]]))

Now we have a joint simulation we can price a worst of or a dual digital

In [40]:
wof = np.minimum(eurusd/1.13-1, 1- audusd/0.69)
wof= np.maximum(wof,0)
wofprice = np.sum(wof)/wof.shape[0]
#apply correct discounting as needed
wofprice,np.sum(np.maximum(eurusd/1.13-1,0))/eurusd.shape[0],\
    np.sum(np.maximum(1-audusd/0.69,0))/audusd.shape[0]


(0.007651947581397627, 0.03997192430892559, 0.03998276965849625)

In [41]:
1/np.sqrt(2*3.141592)

0.3989423219002315